다대다 RNN은 대표적으로 품사 태깅, 개체명 인식 등에서 사용

# 

# *Char RNN*

RNN 입출력 단위가 단어 레벨(word-level)이 아니라 문자 레벨(character-level)로 하여 RNN을 구현한다면, 이를 <b>문자 단위 RNN</b>라고 함.<br>
<u>RNN 구조 자체가 달라진 것은 아니고, 입,출력의 단위가 문자로 바뀜</u><br><br>
<b>문자 단위 RNN을 다대다 구조</b>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

### *processing*

문자 시퀀스 apple을 입력받아 pple!를 출력하는 RNN 구현

In [2]:
input_str = 'apple'
label_str = 'pple!'

In [3]:
char_vocab = sorted(list(set(input_str + label_str)))

In [4]:
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

문자 집합의 크기 : 5


# 

# *Encoding*

In [5]:
char_to_index = {v: k for k, v in enumerate(char_vocab)}
index_to_char = {k: v for k, v in enumerate(char_vocab)}
print(f"char_to_index : {char_to_index}")
print(f"index_to_char : {index_to_char}")

char_to_index : {'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}
index_to_char : {0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [6]:
x_data = [char_to_index[i] for i in input_str]
y_data = [char_to_index[i] for i in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


### *batch 차원 추가*

In [7]:
x_data = torch.LongTensor(x_data)
x_data = F.one_hot(x_data, num_classes=vocab_size) # one-hot encoding (LongTensor 필요)
X = x_data.unsqueeze(0).type(torch.FloatTensor)
Y = torch.LongTensor(y_data).unsqueeze(0)

In [8]:
print(X)
print(Y)

tensor([[[0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.]]])
tensor([[4, 4, 3, 2, 0]])


In [9]:
print(f"훈련 데이터 크기 : {X.shape}")
print(f"레이블 크기 : {Y.shape}")

훈련 데이터 크기 : torch.Size([1, 5, 5])
레이블 크기 : torch.Size([1, 5])


# 

# *Train*

In [10]:
input_size = vocab_size
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [11]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [12]:
net = Net(input_size, hidden_size, output_size)
print(net)

Net(
  (rnn): RNN(5, 5, batch_first=True)
  (fc): Linear(in_features=5, out_features=5, bias=True)
)


In [16]:
outputs = net(X)
print(outputs)
print(outputs.shape)

tensor([[[-0.2175, -0.2541,  0.5675, -0.0843,  0.1576],
         [-0.0720, -0.2235,  0.3873, -0.4740,  0.5187],
         [-0.1215, -0.1076,  0.4717, -0.3196,  0.3633],
         [-0.3617, -0.2319,  0.7496, -0.3049,  0.4235],
         [ 0.0344,  0.1267,  0.7013, -0.5676,  0.5601]]],
       grad_fn=<AddBackward0>)
torch.Size([1, 5, 5])


In [17]:
print(outputs.view(-1, input_size)) # 2차원 텐서로 변환
print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환

tensor([[-0.2175, -0.2541,  0.5675, -0.0843,  0.1576],
        [-0.0720, -0.2235,  0.3873, -0.4740,  0.5187],
        [-0.1215, -0.1076,  0.4717, -0.3196,  0.3633],
        [-0.3617, -0.2319,  0.7496, -0.3049,  0.4235],
        [ 0.0344,  0.1267,  0.7013, -0.5676,  0.5601]], grad_fn=<ViewBackward>)
torch.Size([5, 5])


In [18]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [19]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.52280592918396 prediction:  [[2 4 2 2 2]] true Y:  [4, 4, 3, 2, 0] prediction str:  epeee
1 loss:  1.2713689804077148 prediction:  [[4 4 4 4 4]] true Y:  [4, 4, 3, 2, 0] prediction str:  ppppp
2 loss:  1.073270559310913 prediction:  [[4 4 3 4 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pplp!
3 loss:  0.8618305325508118 prediction:  [[4 4 3 4 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pplp!
4 loss:  0.6540454030036926 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pple!
5 loss:  0.47488659620285034 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pple!
6 loss:  0.34092622995376587 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pple!
7 loss:  0.2342413365840912 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pple!
8 loss:  0.15783968567848206 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pple!
9 loss:  0.10820503532886505 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3